In [ ]:
# ============================================================
# Run SQL Server query locally using Windows Authentication
# ============================================================

import pyodbc
import pandas as pd

# --- PARAMETERS ---
SERVER = r"QWVIDBSQLB401.ngquotit.com"      # your SQL Server or instance
DATABASE = "NGCS"                           # target database
DRIVER = "ODBC Driver 17 for SQL Server"    # or "ODBC Driver 18 for SQL Server"
LOAD_TASK_ID = 13449
COMPANY_ISSUER_ID = 2204
OUT_CSV = "mapping_results.csv"

# --- CONNECTION ---
conn_str = (
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    "Trusted_Connection=yes;"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

print("Connecting with connection string:")
print(conn_str)

conn = pyodbc.connect(conn_str)
print("Connected successfully ✅")

# --- SQL QUERY (no DECLARE or SET NOCOUNT) ---
sql = f"""
SELECT
    p.IssuerPlanId,
    s.PolicyNumber,
    cpp.ProductName
FROM dbo.STGPlanMappingFactors s (NOLOCK)
OUTER APPLY (
    SELECT TOP 1 p.*
    FROM dbo.PlanMappingIssuerDetail p (NOLOCK)
    LEFT JOIN dbo.PlanMappingIssuerDetailState ps (NOLOCK)
        ON p.PlanMappingIssuerDetailId = ps.PlanMappingIssuerDetailId
    WHERE
        (p.IssuerPlanName = s.PlanName OR p.IssuerPlanName IS NULL OR p.IssuerPlanName = '')
        AND (p.IssuerPlanId = s.PlanId OR p.IssuerPlanId IS NULL OR p.IssuerPlanId = '')
        AND (p.[Year] = s.[Year] OR p.[Year] IS NULL)
        AND (p.PercentRate = s.RatePercent OR p.PercentRate IS NULL)
        AND (p.CurrencyRate = s.RateDollar OR p.CurrencyRate IS NULL)
        AND (ps.StateCode = s.StateCode OR ps.StateCode IS NULL)
    ORDER BY
        CASE WHEN p.IssuerPlanName = s.PlanName THEN 1 ELSE 0 END +
        CASE WHEN p.IssuerPlanId   = s.PlanId   THEN 1 ELSE 0 END +
        CASE WHEN p.PercentRate    = s.RatePercent THEN 1 ELSE 0 END +
        CASE WHEN p.[Year]         = s.[Year]   THEN 1 ELSE 0 END +
        CASE WHEN p.CurrencyRate   = s.RateDollar THEN 1 ELSE 0 END +
        CASE WHEN ps.StateCode     = s.StateCode THEN 1 ELSE 0 END
        DESC
) p
JOIN dbo.CompanyProduct cp WITH (NOLOCK)
    ON cp.Deleted = 0
   AND cp.CompanyIssuerId = {COMPANY_ISSUER_ID}
JOIN dbo.CompanyProductPlan cpp WITH (NOLOCK)
    ON cpp.Deleted = 0
   AND cpp.CompanyProductId = cp.CompanyProductId
   AND p.CompanyProductPlanId = cpp.CompanyProductPlanId
WHERE s.LoadTaskId = {LOAD_TASK_ID};
"""

# --- EXECUTE AND FETCH ---
df = pd.read_sql(sql, conn)

print(f"✅ Query executed successfully. Rows returned: {len(df)}")
display(df.head(10))

# --- SAVE TO CSV ---
df.to_csv(OUT_CSV, index=False)
print(f"Results saved to: {OUT_CSV}")

conn.close()
print("Connection closed.")
